# NYC Parking Tickets: Violation Rate Prediction

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

from datetime import datetime, timedelta

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# importing libraries

import csv
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [3]:
selected_columns = [
    'Street Code1',
    'Street Code2',
    'Street Code3',
    'Street Name',
    # 'Violation Location',
    'Violation Post Code',
    'Intersecting Street',
    'Violation Precinct',
    'Violation County',
    'Vehicle Body Type',
    'Issue Date',
    # 'Date First Observed',
    'Violation Time',
    #'Days Parking In Effect',
    #'From Hours In Effect',
    #'To Hours In Effect',
    'Violation Code',
]

'''
Genel olarak, "county" (ilçe) daha büyük bir alanı temsil eder. Bir "county", genellikle belirli bir eyaletin sınırları içinde bulunan büyük bir coğrafi bölgeyi kapsar. Bir eyalette birden çok "county" bulunabilir ve her biri kendi yerel yönetimine sahiptir.

Öte yandan, "precinct" (seçim bölgesi) daha küçük bir coğrafi bölgeyi ifade eder. Bu, genellikle seçmenlerin oy kullandığı belirli bir mahalle veya köyü temsil eder. Seçim bölgesi, seçimlerde kullanılan ve oy verme işlemini düzenleyen bir kavramdır.
'''

n_rows_to_read = 100000
file_path = 'Parking_Violations_2022.csv'
df = pd.read_csv(file_path, usecols=selected_columns, nrows=n_rows_to_read) # nrows=n_rows_to_read
df

,Issue Date,Violation Code,Vehicle Body Type,Street Code1,Street Code2,Street Code3,Violation Precinct,Violation Time,Violation County,Street Name,Intersecting Street,Violation Post Code
0,06/25/2021,40,VAN,63430,69230,13490,88,0130A,K,NORTH PORTLAND AVE,NaN,NaN
1,06/25/2021,20,SUBN,13490,40404,40404,88,0225A,K,AUBURN PLACE,NaN,NaN
2,06/17/2021,98,SDN,79430,47130,11750,88,0809P,K,SOUTH PORTLAND AVE,NaN,NaN
3,06/16/2021,98,SDN,53130,23230,23930,83,0605P,K,JEFFERSON ST,NaN,NaN
4,07/04/2021,40,TAXI,81030,23930,40030,83,1058P,K,STANHOPE ST,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,07/02/2021,36,VAN,0,35710,0,0,0140P,MN,NB CENTRAL PARK WEST,@ W 97TH ST,NaN
99996,07/02/2021,36,SUBN,0,0,0,0,0141P,BK,NB NOSTRAND AVE @ AV,E L,NaN
99997,07/02/2021,36,SUBN,0,0,0,0,0141P,BK,EB 37TH ST @ FT HAMI,LTON PKWY,NaN
99998,07/02/2021,36,SUBN,0,0,0,0,0141P,QN,SB QUEENS BLVD @ 87T,H AVE,NaN


In [4]:
df['Violation Precinct'].isna().sum()

0

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Issue Date           100000 non-null  object 
 1   Violation Code       100000 non-null  int64  
 2   Vehicle Body Type    98749 non-null   object 
 3   Street Code1         100000 non-null  int64  
 4   Street Code2         100000 non-null  int64  
 5   Street Code3         100000 non-null  int64  
 6   Violation Precinct   100000 non-null  int64  
 7   Violation Time       99991 non-null   object 
 8   Violation County     98269 non-null   object 
 9   Street Name          99952 non-null   object 
 10  Intersecting Street  61237 non-null   object 
 11  Violation Post Code  0 non-null       float64
dtypes: float64(1), int64(5), object(6)
memory usage: 9.2+ MB


In [6]:
df['Violation Time']

0        0130A
1        0225A
2        0809P
3        0605P
4        1058P
         ...  
99995    0140P
99996    0141P
99997    0141P
99998    0141P
99999    0141P
Name: Violation Time, Length: 100000, dtype: object

In [7]:
df_1 = df.copy()

In [8]:
# import sweetviz as sv
# analyze_report = sv.analyze(df_1)
# analyze_report.show_html('analyze.html', open_browser=False)

In [9]:
df_1['Violation Date'] = pd.to_datetime(df_1['Issue Date'])
df_1['Violation Date (Month)'] = df_1['Violation Date'].dt.month
df_1['Violation Date (Day)'] = df_1['Violation Date'].dt.day
df_1['Violation Date (Year)'] = df_1['Violation Date'].dt.year
df_1 = df_1.drop(columns=['Issue Date'])

In [10]:
df_1

,Violation Code,Vehicle Body Type,Street Code1,Street Code2,Street Code3,Violation Precinct,Violation Time,Violation County,Street Name,Intersecting Street,Violation Post Code,Violation Date,Violation Date (Month),Violation Date (Day),Violation Date (Year)
0,40,VAN,63430,69230,13490,88,0130A,K,NORTH PORTLAND AVE,NaN,NaN,2021-06-25,6,25,2021
1,20,SUBN,13490,40404,40404,88,0225A,K,AUBURN PLACE,NaN,NaN,2021-06-25,6,25,2021
2,98,SDN,79430,47130,11750,88,0809P,K,SOUTH PORTLAND AVE,NaN,NaN,2021-06-17,6,17,2021
3,98,SDN,53130,23230,23930,83,0605P,K,JEFFERSON ST,NaN,NaN,2021-06-16,6,16,2021
4,40,TAXI,81030,23930,40030,83,1058P,K,STANHOPE ST,NaN,NaN,2021-07-04,7,4,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,36,VAN,0,35710,0,0,0140P,MN,NB CENTRAL PARK WEST,@ W 97TH ST,NaN,2021-07-02,7,2,2021
99996,36,SUBN,0,0,0,0,0141P,BK,NB NOSTRAND AVE @ AV,E L,NaN,2021-07-02,7,2,2021
99997,36,SUBN,0,0,0,0,0141P,BK,EB 37TH ST @ FT HAMI,LTON PKWY,NaN,2021-07-02,7,2,2021
99998,36,SUBN,0,0,0,0,0141P,QN,SB QUEENS BLVD @ 87T,H AVE,NaN,2021-07-02,7,2,2021


In [11]:
def convert_to_24hr_format(time_str):
    try:
        if '.' in time_str:
            return None
        if time_str[-1] == 'P':
            return '{:02d}:{:02d}'.format((int(time_str[:2]) + 12) % 24, int(time_str[2:4]))
        elif time_str[-1] == 'A':
            return '{:02d}:{:02d}'.format(int(time_str[:2]) % 12, int(time_str[2:4]))
        else:
            return None
    except (ValueError, TypeError):
        return None

df_1['Violation Time'] = df_1['Violation Time'].astype(str)
df_1.loc[:, 'Violation Time'] = df_1['Violation Time'].apply(convert_to_24hr_format)
df_1 = df_1.dropna(subset=['Violation Time'])

In [12]:
df_1.loc[:, 'Violation Time'] = pd.to_datetime(df_1['Violation Time'], format='%H:%M', errors='coerce').dt.time

In [13]:
df_1['Violation Time']

0        01:30:00
1        02:25:00
2        20:09:00
3        18:05:00
4        22:58:00
           ...   
99995    13:40:00
99996    13:41:00
99997    13:41:00
99998    13:41:00
99999    13:41:00
Name: Violation Time, Length: 99983, dtype: object

In [14]:
df_1

,Violation Code,Vehicle Body Type,Street Code1,Street Code2,Street Code3,Violation Precinct,Violation Time,Violation County,Street Name,Intersecting Street,Violation Post Code,Violation Date,Violation Date (Month),Violation Date (Day),Violation Date (Year)
0,40,VAN,63430,69230,13490,88,01:30:00,K,NORTH PORTLAND AVE,NaN,NaN,2021-06-25,6,25,2021
1,20,SUBN,13490,40404,40404,88,02:25:00,K,AUBURN PLACE,NaN,NaN,2021-06-25,6,25,2021
2,98,SDN,79430,47130,11750,88,20:09:00,K,SOUTH PORTLAND AVE,NaN,NaN,2021-06-17,6,17,2021
3,98,SDN,53130,23230,23930,83,18:05:00,K,JEFFERSON ST,NaN,NaN,2021-06-16,6,16,2021
4,40,TAXI,81030,23930,40030,83,22:58:00,K,STANHOPE ST,NaN,NaN,2021-07-04,7,4,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,36,VAN,0,35710,0,0,13:40:00,MN,NB CENTRAL PARK WEST,@ W 97TH ST,NaN,2021-07-02,7,2,2021
99996,36,SUBN,0,0,0,0,13:41:00,BK,NB NOSTRAND AVE @ AV,E L,NaN,2021-07-02,7,2,2021
99997,36,SUBN,0,0,0,0,13:41:00,BK,EB 37TH ST @ FT HAMI,LTON PKWY,NaN,2021-07-02,7,2,2021
99998,36,SUBN,0,0,0,0,13:41:00,QN,SB QUEENS BLVD @ 87T,H AVE,NaN,2021-07-02,7,2,2021


In [15]:
df_1.sort_values(by=['Violation Date', 'Violation Time'], inplace=True)
df_1

,Violation Code,Vehicle Body Type,Street Code1,Street Code2,Street Code3,Violation Precinct,Violation Time,Violation County,Street Name,Intersecting Street,Violation Post Code,Violation Date,Violation Date (Month),Violation Date (Day),Violation Date (Year)
6647,20,SUBN,72720,27180,29220,46,00:04:00,BX,WALTON AVE,NaN,NaN,2000-06-27,6,27,2000
3082,78,NaN,0,40404,40404,105,22:00:00,Q,BROOKVILL,NaN,NaN,2000-06-28,6,28,2000
5337,35,VAN,10610,40404,40404,14,00:52:00,NY,7 AVE,NaN,NaN,2000-07-15,7,15,2000
15310,67,SDN,68390,48290,54090,104,09:10:00,NaN,WOODWARD AVE,NaN,NaN,2010-07-10,7,10,2010
1635,98,VAN,19990,17540,17740,106,01:05:00,Q,107 AVE,NaN,NaN,2010-07-21,7,21,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7140,14,SUBN,62200,33490,0,100,15:31:00,Q,SHORE FRONT PKWY,BEACH 100 ST,NaN,2022-07-20,7,20,2022
3717,74,MOPE,31250,27950,47530,121,04:30:00,R,GROVE AVE,NaN,NaN,2024-07-24,7,24,2024
3730,74,MOPE,31250,27950,47530,121,04:30:00,R,GROVE AVE,NaN,NaN,2024-07-24,7,24,2024
9132,67,SDN,0,0,0,71,08:51:00,K,W/S BROOKLYN AVE,S/O STERLING ST,NaN,2029-06-29,6,29,2029


In [16]:
start_date = pd.to_datetime('2021-07-01')
end_date = pd.to_datetime('2022-06-30')

In [17]:
df_1 = df_1[(df_1['Violation Date'] >= start_date) & (df_1['Violation Date'] <= end_date)]
df_1

,Violation Code,Vehicle Body Type,Street Code1,Street Code2,Street Code3,Violation Precinct,Violation Time,Violation County,Street Name,Intersecting Street,Violation Post Code,Violation Date,Violation Date (Month),Violation Date (Day),Violation Date (Year)
19777,21,4DSD,0,0,0,107,00:00:00,Qns,Opposite of 70-87 PA,RK DRIVE EAST,NaN,2021-07-01,7,1,2021
22002,0,VAN,0,0,0,0,00:00:00,NaN,NaN,NaN,NaN,2021-07-01,7,1,2021
23604,0,SUBN,0,0,0,0,00:00:00,NaN,NaN,NaN,NaN,2021-07-01,7,1,2021
26384,21,4DSD,8120,25860,25890,40,00:00:00,Bronx,ALEXANDER AVENUE,NaN,NaN,2021-07-01,7,1,2021
28018,21,SUBN,0,0,0,78,00:00:00,Kings,Front Of 190 12 STRE,ET,NaN,2021-07-01,7,1,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14316,20,SDN,13820,0,0,84,20:12:00,K,BROOKLYN BRIDGE PARK,PIER 5 LOOP,NaN,2021-08-16,8,16,2021
10511,41,SUBN,32560,49130,13430,81,08:45:00,K,DEWEY PLACE,NaN,NaN,2021-08-17,8,17,2021
10081,20,SUBN,0,30140,0,101,17:08:00,Q,SEAGIRT ST,B 17 ST,NaN,2021-08-31,8,31,2021
10496,41,SUBN,0,0,0,81,08:55:00,K,C/O DEWEY PL,HERKIMER,NaN,2021-09-24,9,24,2021


In [18]:
df_1['Violation Hour'] = df_1['Violation Time'].apply(lambda x: (x.hour % 24)).astype(int)
df_1['Violation Hour']

19777     0
22002     0
23604     0
26384     0
28018     0
         ..
14316    20
10511     8
10081    17
10496     8
14226    18
Name: Violation Hour, Length: 50776, dtype: int32

In [28]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50776 entries, 19777 to 14226
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Violation Code           50776 non-null  int64         
 1   Vehicle Body Type        50189 non-null  object        
 2   Street Code1             50776 non-null  int64         
 3   Street Code2             50776 non-null  int64         
 4   Street Code3             50776 non-null  int64         
 5   Violation Precinct       50776 non-null  int64         
 6   Violation Time           50776 non-null  object        
 7   Violation County         49652 non-null  object        
 8   Street Name              50732 non-null  object        
 9   Intersecting Street      19159 non-null  object        
 10  Violation Post Code      0 non-null      float64       
 11  Violation Date           50776 non-null  datetime64[ns]
 12  Violation Date (Month)   50776 no

In [19]:
def calculate_location_violation_rate(df, location, time_range, precinct, violation_date):
    start_time = datetime.strptime(time_range[0], "%H:%M").time()
    end_time = datetime.strptime(time_range[1], "%H:%M").time()

    # filtered_data = df[(df['Street Name'] == location) & (df['Violation Date'] == violation_date) & (~df['Violation Time'].isna()) &
    #                    (df['Violation Time'].apply(lambda x: not pd.isna(x) and x == x and start_time <= x <= end_time))]

    filtered_data = df[(df['Street Name'] == location) &
                   (df['Violation Date'] == violation_date) &
                   (~df['Violation Time'].isna()) &
                   (df['Violation Time'].apply(lambda x: start_time <= x <= end_time if pd.notna(x) else False))]

    print("filtered data:", len(filtered_data))

    total_location_violations = len(df_1[(df_1['Violation Time'].between(start_time, end_time)) &
                                      (df_1['Violation Precinct'] == precinct) &
                                      (df_1['Violation Date'] == violation_date)])

    print("total_location_violations:", total_location_violations)

    if total_location_violations == 0:
        return 0

    location_violations = len(filtered_data)

    location_violation_rate = (location_violations / total_location_violations) * 100

    return location_violation_rate

location = 'GRANDVIEW AVE'
location_precinct = 121
time_range = ('00:00', '13:00')
violation_date = pd.to_datetime('2021-07-25')

rate = calculate_location_violation_rate(df_1, location, time_range, location_precinct, violation_date)
print(f'Location Violation Rate for {location} between {time_range[0]} and {time_range[1]}: {rate:.2f}%')

filtered data: 4
total_location_violations: 5
Location Violation Rate for GRANDVIEW AVE between 00:00 and 13:00: 80.00%


In [20]:
# import dask.dataframe as dd
# from dask.distributed import Client
# from datetime import datetime, timedelta
#
# ddf = dd.from_pandas(df_1, npartitions=10)
#
# client = Client()
#
# def calculate_violation_rate(row):
#     location = row['Street Name']
#     hour = row['Violation Hour']
#     violation_date = row['Violation Date']
#     precinct = row['Violation Precinct']
#
#     if hour == 23:
#         start_time = datetime.strptime('23:00', '%H:%M').time()
#         end_time = datetime.strptime('23:59', '%H:%M').time()
#     else:
#         start_time = datetime.strptime(f'{hour:02d}:00', '%H:%M').time()
#         end_time = (datetime.strptime(f'{hour+1:02d}:00', '%H:%M') + timedelta(minutes=59, seconds=59)).time()
#
#     total_location_violations = len(ddf[(ddf['Violation Time'].between(start_time, end_time)) &
#                                         (ddf['Violation Precinct'] == precinct) &
#                                         (ddf['Violation Date'] == violation_date)])
#
#     location_violations = len(ddf[(ddf['Street Name'] == location) &
#                                    (ddf['Violation Time'].between(start_time, end_time)) &
#                                    (ddf['Violation Date'] == violation_date)])
#
#     if total_location_violations == 0:
#         return 0
#
#     location_violation_rate = (location_violations / total_location_violations) * 100
#
#     return location_violation_rate
#
# ddf['Violation Rate'] = ddf.apply(calculate_violation_rate, axis=1, meta=('x', 'f8'))
#
# result_with_violation_rate = ddf.compute()
#
# client.close()
# result_with_violation_rate

In [21]:
def calculate_violation_rate(row):
    location = row['Street Name']
    hour = row['Violation Hour']
    violation_date = row['Violation Date']
    precinct = row['Violation Precinct']

    # Handle the case where hour is 24
    if hour == 23:
        start_time = datetime.strptime('23:00', '%H:%M').time()
        end_time = datetime.strptime('23:59', '%H:%M').time()
    else:
        start_time = datetime.strptime(f'{hour:02d}:00', '%H:%M').time()
        end_time = (datetime.strptime(f'{hour+1:02d}:00', '%H:%M') + timedelta(minutes=59, seconds=59)).time()


    total_location_violations = len(df_1[(df_1['Violation Time'].between(start_time, end_time)) &
                                      (df_1['Violation Precinct'] == precinct) &
                                      (df_1['Violation Date'] == violation_date)])

    location_violations = len(df_1[(df_1['Street Name'] == location) &
                                 (df_1['Violation Time'].between(start_time, end_time)) &
                                 (df_1['Violation Date'] == violation_date)])

    if total_location_violations == 0:
        return 0

    location_violation_rate = (location_violations / total_location_violations) * 100

    return location_violation_rate

if 'Violation Rate' in df_1:
    df_1.drop('Violation Rate', axis=1, inplace=True)

df_1['Violation Rate'] = df_1.apply(lambda row: calculate_violation_rate(row), axis=1)

In [22]:
df_1['Violation Rate']

19777     50.000000
22002      0.000000
23604      0.000000
26384     20.833333
28018    100.000000
            ...    
14316    100.000000
10511    100.000000
10081    100.000000
10496    100.000000
14226    100.000000
Name: Violation Rate, Length: 50776, dtype: float64

In [23]:
df_1

,Violation Code,Vehicle Body Type,Street Code1,Street Code2,Street Code3,Violation Precinct,Violation Time,Violation County,Street Name,Intersecting Street,Violation Post Code,Violation Date,Violation Date (Month),Violation Date (Day),Violation Date (Year),Violation Hour,Violation Rate
19777,21,4DSD,0,0,0,107,00:00:00,Qns,Opposite of 70-87 PA,RK DRIVE EAST,NaN,2021-07-01,7,1,2021,0,50.000000
22002,0,VAN,0,0,0,0,00:00:00,NaN,NaN,NaN,NaN,2021-07-01,7,1,2021,0,0.000000
23604,0,SUBN,0,0,0,0,00:00:00,NaN,NaN,NaN,NaN,2021-07-01,7,1,2021,0,0.000000
26384,21,4DSD,8120,25860,25890,40,00:00:00,Bronx,ALEXANDER AVENUE,NaN,NaN,2021-07-01,7,1,2021,0,20.833333
28018,21,SUBN,0,0,0,78,00:00:00,Kings,Front Of 190 12 STRE,ET,NaN,2021-07-01,7,1,2021,0,100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14316,20,SDN,13820,0,0,84,20:12:00,K,BROOKLYN BRIDGE PARK,PIER 5 LOOP,NaN,2021-08-16,8,16,2021,20,100.000000
10511,41,SUBN,32560,49130,13430,81,08:45:00,K,DEWEY PLACE,NaN,NaN,2021-08-17,8,17,2021,8,100.000000
10081,20,SUBN,0,30140,0,101,17:08:00,Q,SEAGIRT ST,B 17 ST,NaN,2021-08-31,8,31,2021,17,100.000000
10496,41,SUBN,0,0,0,81,08:55:00,K,C/O DEWEY PL,HERKIMER,NaN,2021-09-24,9,24,2021,8,100.000000


In [24]:
def scale_between_0_and_1(value, min_value, max_value):
    """
    Scale a value to be between 0 and 1.

    :param value: The value to be scaled.
    :param min_value: The minimum value in the range of your data.
    :param max_value: The maximum value in the range of your data.
    :return: Scaled value between 0 and 1.
    """
    if max_value == min_value:
        return 0  # Avoid division by zero if all values are the same

    return (value - min_value) / (max_value - min_value)

In [25]:
df_1['Violation Rate (Scaled)'] = df_1['Violation Rate'].apply(lambda x: scale_between_0_and_1(x, df_1['Violation Rate'].min(), df_1['Violation Rate'].max()))
df_1['Violation Rate (Scaled)']

19777    0.018519
22002    0.000000
23604    0.000000
26384    0.007716
28018    0.037037
           ...   
14316    0.037037
10511    0.037037
10081    0.037037
10496    0.037037
14226    0.037037
Name: Violation Rate (Scaled), Length: 50776, dtype: float64

In [26]:
df_1['Violation Rate'].max()

2700.0